In [6]:
import pandas as pd

# Đọc file gốc
df = pd.read_csv('h:/SentimentAnalystSchool/MidtermExam/Semeval/df_train_all.csv')

print(f"Đã đọc {len(df):,} dòng dữ liệu")
print(f"\nCác cột: {df.columns.tolist()}")
print(f"\nPhân bố polarity:")
print(df['polarity'].value_counts())

Đã đọc 5,416 dòng dữ liệu

Các cột: ['domain', 'review_id', 'sentence_id', 'text', 'aspect', 'polarity', 'label']

Phân bố polarity:
polarity
positive    3294
negative    1833
neutral      289
Name: count, dtype: int64


In [7]:
# ============================================
# CẤU HÌNH SỐ LƯỢNG MẪU THEO SENTIMENT
# ============================================
# Chỉnh sửa các giá trị bên dưới để thay đổi số lượng mẫu cho mỗi nhãn
# Đặt None nếu muốn lấy tất cả dữ liệu của nhãn đó

BALANCE_CONFIG = {
    'positive': 500,   # Số lượng mẫu positive tối đa
    'negative': 500,   # Số lượng mẫu negative tối đa  
    'neutral': 289     # Số lượng mẫu neutral tối đa
}

# Đặt BALANCE_CONFIG = None nếu muốn giữ tất cả dữ liệu
# BALANCE_CONFIG = None

# ============================================

# Tạo DataFrame với 2 cột: reviewText và Sentiment
df_export = pd.DataFrame({
    'reviewText': df['text'],
    'Sentiment': df['polarity']
})

print(f"Tổng số mẫu ban đầu: {len(df_export):,}")
print(f"\nPhân bố Sentiment ban đầu:")
print(df_export['Sentiment'].value_counts())

Tổng số mẫu ban đầu: 5,416

Phân bố Sentiment ban đầu:
Sentiment
positive    3294
negative    1833
neutral      289
Name: count, dtype: int64


In [8]:
# Áp dụng cân bằng dữ liệu nếu có BALANCE_CONFIG
if BALANCE_CONFIG:
    print("=" * 60)
    print("ÁP DỤNG CÂN BẰNG DỮ LIỆU")
    print("=" * 60)
    
    balanced_dfs = []
    for sentiment, max_count in BALANCE_CONFIG.items():
        sentiment_df = df_export[df_export['Sentiment'] == sentiment]
        current_count = len(sentiment_df)
        
        if max_count is not None and current_count > max_count:
            sentiment_df = sentiment_df.sample(n=max_count, random_state=42)
            print(f"  {sentiment}: {current_count:,} → {max_count:,} (giảm {current_count - max_count:,})")
        else:
            print(f"  {sentiment}: {current_count:,} (giữ nguyên)")
        
        balanced_dfs.append(sentiment_df)
    
    df_final = pd.concat(balanced_dfs, ignore_index=True)
    # Shuffle lại
    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
else:
    df_final = df_export

print(f"\n📊 Tổng số mẫu sau khi xử lý: {len(df_final):,}")
print(f"\n📈 Phân bố Sentiment cuối cùng:")
print(df_final['Sentiment'].value_counts())

ÁP DỤNG CÂN BẰNG DỮ LIỆU
  positive: 3,294 → 500 (giảm 2,794)
  negative: 1,833 → 500 (giảm 1,333)
  neutral: 289 (giữ nguyên)

📊 Tổng số mẫu sau khi xử lý: 1,289

📈 Phân bố Sentiment cuối cùng:
Sentiment
positive    500
negative    500
neutral     289
Name: count, dtype: int64


In [9]:
# Export ra file CSV
output_path = 'h:/SentimentAnalystSchool/MidtermExam/Semeval/semeval_500_500_289.csv'
df_final.to_csv(output_path, index=False)

print(f"✓ Đã export {len(df_final):,} dòng ra file: {output_path}")
print(f"\nMẫu dữ liệu đầu tiên:")
df_final.head(10)

✓ Đã export 1,289 dòng ra file: h:/SentimentAnalystSchool/MidtermExam/Semeval/semeval_500_500_289.csv

Mẫu dữ liệu đầu tiên:


,reviewText,Sentiment
0,The keyboard is a little wonky with having to ...,neutral
1,I'm still exploring the various features but o...,neutral
2,Just what the doctor ordered.,positive
3,Quite simply this is the best laptop I have ev...,positive
4,The farro salad and the mashed yukon potatoes ...,positive
5,I highly recommend it.,positive
6,"I love my Apple, it is quick and easy to use.",positive
7,"while the keyboard itself is alright, the plat...",neutral
8,This laptop is very large and barely fits in a...,negative
9,The cover is a soft rubber texture without the...,positive


## Benchmark với Model DistilBERT + LSTM

Sử dụng model đã train để benchmark trên bộ dữ liệu Semeval

In [ ]:
# Import các thư viện cần thiết cho benchmark
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Định nghĩa Model Architecture (phải giống với model đã train)
class DistilBertLSTMClassifier(nn.Module):
    def __init__(self, n_classes=3, lstm_hidden_size=128, lstm_layers=2, dropout=0.3):
        super(DistilBertLSTMClassifier, self).__init__()
        
        # DistilBERT
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        
        # LSTM
        self.lstm = nn.LSTM(
            input_size=768,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=dropout if lstm_layers > 1 else 0,
            bidirectional=True
        )
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Classifier
        self.fc = nn.Linear(lstm_hidden_size * 2, n_classes)
        
    def forward(self, input_ids, attention_mask):
        distilbert_output = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        sequence_output = distilbert_output.last_hidden_state
        lstm_output, (hidden, cell) = self.lstm(sequence_output)
        hidden_fwd = hidden[-2, :, :]
        hidden_bwd = hidden[-1, :, :]
        hidden_concat = torch.cat((hidden_fwd, hidden_bwd), dim=1)
        hidden_concat = self.dropout(hidden_concat)
        output = self.fc(hidden_concat)
        return output

print("✓ Model architecture defined")

In [ ]:
# Dataset class cho benchmark
class BenchmarkDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = str(self.data.loc[idx, 'text'])
        aspect = str(self.data.loc[idx, 'aspect'])
        label = self.data.loc[idx, 'label']
        
        # Kết hợp text và aspect
        combined_text = text + " [SEP] " + aspect
        
        encoding = self.tokenizer.encode_plus(
            combined_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

print("✓ Dataset class defined")

In [ ]:
# Load model weights
MODEL_PATH = 'h:/SentimentAnalystSchool/MidtermExam/ModelWeight/distilbert_lstm_model_complete.pt'

# Load checkpoint
checkpoint = torch.load(MODEL_PATH, map_location=device)

# Khởi tạo model
model = DistilBertLSTMClassifier(n_classes=3)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

# Load label encoder từ checkpoint
label_encoder = checkpoint['label_encoder']
MAX_LEN = checkpoint.get('max_len', 128)

print("✓ Model loaded successfully")
print(f"Label classes: {label_encoder.classes_}")

In [ ]:
# Load và chuẩn bị dữ liệu benchmark
df_benchmark = pd.read_csv('h:/SentimentAnalystSchool/MidtermExam/Semeval/df_train_all.csv')

print(f"Loaded {len(df_benchmark):,} samples for benchmark")
print(f"\nPolarity distribution:")
print(df_benchmark['polarity'].value_counts())

# Encode labels (sử dụng label encoder từ model)
# Map polarity sang label number
polarity_to_label = {label: idx for idx, label in enumerate(label_encoder.classes_)}
df_benchmark['label'] = df_benchmark['polarity'].map(polarity_to_label)

# Kiểm tra xem có giá trị nào không map được không
missing_labels = df_benchmark['label'].isna().sum()
if missing_labels > 0:
    print(f"\n⚠ Warning: {missing_labels} samples có polarity không khớp với label encoder")
    print(f"Polarity trong data: {df_benchmark['polarity'].unique()}")
    print(f"Labels trong model: {label_encoder.classes_}")

# Bỏ các dòng không map được
df_benchmark = df_benchmark.dropna(subset=['label'])
df_benchmark['label'] = df_benchmark['label'].astype(int)

print(f"\nSamples sau khi xử lý: {len(df_benchmark):,}")

In [ ]:
# Tạo DataLoader cho benchmark
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
BATCH_SIZE = 16

benchmark_dataset = BenchmarkDataset(df_benchmark, tokenizer, MAX_LEN)
benchmark_loader = DataLoader(benchmark_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"✓ DataLoader created with {len(benchmark_loader)} batches")

In [ ]:
# Hàm benchmark
def run_benchmark(model, data_loader, device):
    """
    Chạy benchmark trên bộ dữ liệu
    
    Returns:
        accuracy, all_preds, all_labels
    """
    model.eval()
    all_preds = []
    all_labels = []
    
    print("=" * 60)
    print("RUNNING BENCHMARK")
    print("=" * 60)
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            if (batch_idx + 1) % 50 == 0:
                print(f"  Processed {batch_idx + 1}/{len(data_loader)} batches...")
    
    accuracy = accuracy_score(all_labels, all_preds)
    
    return accuracy, all_preds, all_labels

print("✓ Benchmark function defined")

In [ ]:
# Chạy benchmark
import time

start_time = time.time()
accuracy, all_preds, all_labels = run_benchmark(model, benchmark_loader, device)
elapsed_time = time.time() - start_time

print(f"\n" + "=" * 60)
print("BENCHMARK RESULTS")
print("=" * 60)
print(f"\n⏱ Time: {elapsed_time:.2f} seconds")
print(f"📊 Total samples: {len(all_labels):,}")
print(f"🎯 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

In [ ]:
# Chi tiết classification report
from sklearn.metrics import classification_report

label_names = ['Negative', 'Neutral', 'Positive']
report = classification_report(all_labels, all_preds, target_names=label_names)
print("📋 Classification Report:")
print("-" * 60)
print(report)

In [ ]:
# Confusion Matrix Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Tạo confusion matrix
cm = confusion_matrix(all_labels, all_preds)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Confusion Matrix với số lượng
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_names, yticklabels=label_names, ax=axes[0])
axes[0].set_title('Confusion Matrix (Counts)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# 2. Confusion Matrix với phần trăm (normalized)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
sns.heatmap(cm_normalized, annot=True, fmt='.1f', cmap='Blues',
            xticklabels=label_names, yticklabels=label_names, ax=axes[1])
axes[1].set_title('Confusion Matrix (Percentage %)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.savefig('benchmark_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💾 Confusion matrix saved to 'benchmark_confusion_matrix.png'")

In [ ]:
# Thống kê chi tiết theo từng class
print("📊 Detailed Statistics per Class:")
print("-" * 60)

for i, label in enumerate(label_names):
    total = cm[i].sum()
    correct = cm[i][i]
    incorrect = total - correct
    class_acc = (correct / total * 100) if total > 0 else 0
    
    print(f"\n{label.upper()}:")
    print(f"  • Total samples: {total:,}")
    print(f"  • Correct predictions: {correct:,} ({class_acc:.1f}%)")
    print(f"  • Incorrect predictions: {incorrect:,} ({100-class_acc:.1f}%)")

# Summary
print("\n" + "=" * 60)
print("📈 OVERALL SUMMARY:")
print(f"  • Dataset: Semeval (df_train_all.csv)")
print(f"  • Model: DistilBERT + LSTM")
print(f"  • Weight file: distilbert_lstm_model_complete.pt")
print(f"  • Overall Accuracy: {accuracy*100:.2f}%")
print("=" * 60)